In [2]:
import pandas as pd

AttributeError: module 'pandas' has no attribute 'compat'

In [ ]:
diabetes = pd.read_csv('pima-indians-diabetes.csv')

In [ ]:
diabetes.head()

In [ ]:
# now let's start cleaing up the data

In [ ]:
diabetes.columns

In [ ]:
cols_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']

In [ ]:
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x:(x-x.min())/(x.max()-x.min()))

In [ ]:
diabetes[cols_to_norm].head()

In [ ]:
diabetes.head()

In [ ]:
import tensorflow as tf

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
# we are creating a numeric feature column herw
num_preg = tf.feature_column.numeric_column('Number_pregnant')
plasma_gluc = tf.feature_column.numeric_column('Glucose_concentration')
dias_press = tf.feature_column.numeric_column('Blood_pressure')
tricep = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
diabetes_pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')

In [ ]:
#categorical feature column
assigned_group= tf.feature_column.categorical_column_with_vocabulary_list('Group',['A','B','C','D'])

In [ ]:
# now for hash bucket 
#assigned_group = tf.feature_column.categorical_column_with_hash_bucket('Group',hash_bucket_size=10)

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
diabetes['Age'].hist(bins = 20)

In [ ]:
# now we are taking the age with a conitnious value to categorical value 
age_bucket = tf.feature_column.bucketized_column(age,boundaries=[20,30,40,50,60,70,80])

In [ ]:
age_bucket

In [ ]:
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,assigned_group, age_bucket]

In [ ]:
# TRAIN TEST SPLIT
x_data = diabetes.drop('Class',axis=1)

In [ ]:
labels= diabetes['Class']

In [ ]:

x_data.head()


In [ ]:
labels.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.3, random_state=101)

In [ ]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,
                                                        batch_size = 10,num_epochs=1000,shuffle = True)

In [ ]:
# we are using classifaction so let;s do linear classifer
model  = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

In [ ]:
model.train(input_fn=input_func,steps=1000)

In [ ]:
# now let's start evaluating the model
eval_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,y= y_test,batch_size=10,
                                                                num_epochs=1,shuffle=False)

In [ ]:
results = model.evaluate(eval_input_func)

In [ ]:
results

In [ ]:
# well we got around 74% accuracy let's start with predictions
pred_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [ ]:
predictios = model.predict(pred_input_func)

In [ ]:
my_pred = list(predictios)

In [ ]:
# dnn if we want 3 layers of 10 neurons each
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
#we can't simply train DNN similar to Linearclassifer

In [ ]:
embedded_group_col = tf.feature_column.embedding_column(assigned_group,dimension=4)

In [ ]:
feat_cols = [num_preg ,plasma_gluc,dias_press ,tricep ,insulin,bmi,diabetes_pedigree ,embedded_group_col, age_bucket]

In [ ]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y = y_train,num_epochs=1000,batch_size=10,shuffle=True)

In [ ]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

In [ ]:
dnn_model.train(input_fn=input_func,steps=1000)

In [ ]:
#let's evaluate the dnnmodel
eval_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,y = y_test,
                                                                batch_size=10,num_epochs=1,shuffle=False)

In [ ]:
dnn_model.evaluate(input_fn=eval_input_func)